# SageMaker built-in Latent Dirichlet Allocation(LDA) example

1. [Introduction](#Introduction)  
2. [Development Environment and Data Preparation](#Development-Environment-and-Data-Preparation)
    1. [Installation](#Installation)  
    2. [Data Preparation](#Data-Preparation)    
3. [Text Preprocessing](#Text-Preprocessing)  
    1. [Tokenization](#Tokenization)  
    2. [Create Corpus and Vocabulary](#Create-Corpus-and-Vocabulary)   
4. [Training the LDA Model](#Training-the-LDA-Model)  
    1. [Create LDA Container](#Create-LDA-Container)  
    2. [Set Hyperparameters¶](#Set-Hyperparameters¶)   
    3. [Training](#Training)
5. [Evaluating the Model Output](#Evaluating-the-Model-Output)  
    1. [Interpretation](#Interpretation)  
    2. [Deploy and Inference](#Deploy-and-Inference)  

# Introduction

このnotebookはSageMakerのビルトインアルゴリズムの一つであるLatent Dirichlet Allocation(LDA)のサンプルです。    
ビルトインアルゴリズムを使用する場合、学習とデプロイに関連するコードのほとんどを開発者が意識する必要がなくなる利点があります。    

データはlivedoor ニュースコーパスを使用します。

NOTE: このデモは、SagemakerNotebookインスタンスで動作検証しています

# Development Environment and Data Preparation

## Installation
このNotebookはSageMakerのconda_mxnet_p36カーネルを利用しています。    
日本語処理のため、[GiNZA](https://megagonlabs.github.io/ginza/)などをインストールします。    

_**NOTE: 日本語処理はmecabを使用するなど開発者の好みに変更することができます**_  

In [ ]:
!pip install ginza==4.0.6
!pip install mojimoji neologdn

In [ ]:
%matplotlib inline

import os
import time
import re
import tarfile
import json

# visualization
import seaborn as sns

import mxnet as mx
import numpy as np

# Amazon Web Services (AWS) SDK for Python
import boto3

# SageMaker Python SDK
import sagemaker
from sagemaker.amazon.common import RecordSerializer
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker import get_execution_role

## Data Preparation

株式会社ロンウイットさんで公開している[livedoor ニュースコーパス](https://www.rondhuit.com/download.html)をダウンロードします。

In [ ]:
# https://radiology-nlp.hatenablog.com/entry/2019/11/25/124219

!mkdir dataset
!cd dataset
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxvf ldcc-20140209.tar.gz

!echo -e "filename\tarticle"$(for category in $(basename -a `find ./text -type d` | grep -v text | sort); do echo -n "\t"; echo -n $category; done) > ./text/livedoor.tsv

!for filename in `basename -a ./text/dokujo-tsushin/dokujo-tsushin-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/dokujo-tsushin/$filename`; echo -e "\t1\t0\t0\t0\t0\t0\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/it-life-hack/it-life-hack-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/it-life-hack/$filename`; echo -e "\t0\t1\t0\t0\t0\t0\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/kaden-channel/kaden-channel-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/kaden-channel/$filename`; echo -e "\t0\t0\t1\t0\t0\t0\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/livedoor-homme/livedoor-homme-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/livedoor-homme/$filename`; echo -e "\t0\t0\t0\t1\t0\t0\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/movie-enter/movie-enter-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/movie-enter/$filename`; echo -e "\t0\t0\t0\t0\t1\t0\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/peachy/peachy-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/peachy/$filename`; echo -e "\t0\t0\t0\t0\t0\t1\t0\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/smax/smax-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/smax/$filename`; echo -e "\t0\t0\t0\t0\t0\t0\t1\t0\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/sports-watch/sports-watch-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/sports-watch/$filename`; echo -e "\t0\t0\t0\t0\t0\t0\t0\t1\t0"; done >> ./text/livedoor.tsv
!for filename in `basename -a ./text/topic-news/topic-news-*`; do echo -n "$filename"; echo -ne "\t"; echo -n `sed -e '1,3d' ./text/topic-news/$filename`; echo -e "\t0\t0\t0\t0\t0\t0\t0\t0\t1"; done >> ./text/livedoor.tsv

In [ ]:
import pandas as pd

df = pd.read_csv('./text/livedoor.tsv', sep='\t')
df = df.sample(df.shape[0], random_state=42).reset_index(drop=True)
print(df.shape)
df.head()

# Text Preprocessing

## Tokenization

ここではテキストを意味のある単位で分割していきたいのですが、日本語は英語とは異なり、各単語があらかじめスペースで区切られていません。    
次のセルでは日本語NLPライブラリのGiNZAを使って文章を分割していきます。    

また以下の処理     

- URLの除去
- htmlタグを除去
- 文字の正規化、全角を半角に統一

を行って、このサンプルでは名詞と形容詞のみを抽出します。

In [ ]:
import spacy

from bs4 import BeautifulSoup
import re
import mojimoji
import neologdn


nlp = spacy.load('ja_ginza', disable=['ner'])
stop_words = spacy.lang.ja.stop_words.STOP_WORDS


def filterHtmlTag(txt):
    soup = BeautifulSoup(txt, 'html.parser')
    txt = soup.get_text(strip=True)
    return txt


def normalize_text(text):
    result = mojimoji.zen_to_han(text, kana=False)
    result = neologdn.normalize(result)
    return result


def text_to_words(text):
    
    basic_words = []
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text) # URLの除去
    text = filterHtmlTag(text) # htmlタグの除去
    text = normalize_text(text) # 正規化
    doc = nlp(text)
    
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ in stop_words:
                continue
            
            # 形容詞の原型を取得
            elif token.pos_ in ('ADJ'):
                basic_words.append(token.lemma_)
                
            # 名詞を取得
            elif token.pos_ in ('NOUN'):
                basic_words.append(token.orth_)
        
    basic_words = ' '.join(basic_words)
    return basic_words

In [ ]:
text_to_words(df.article[0])

In [ ]:
%%time
# m5.xlargeで約7minかかります
from multiprocessing import Pool

with Pool() as p:
    docs = p.map(func=text_to_words, iterable=df.article)

In [ ]:
docs[0:5]

## Create Corpus and Vocabulary

LDAの学習のためにコーパスと辞書を作成します。    
- コーパスは各記事を単語の頻度表現（Bag of Words）にしたものです。    
- 辞書は重複のない単語のリストです。    

どちらもScikit-learnの`CountVectorizer`を使用して作成します。    

In [ ]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizerの設定
NGRAM=1
MAX_DF=0.95
MIN_DF=0.01
NUM_VOCAB=None

count_vec = CountVectorizer(ngram_range=(1, NGRAM), max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_VOCAB)
count_vec = count_vec.fit(docs)
bags_of_words = count_vec.transform(docs).toarray()

In [ ]:
with open(f"count_vec.pkl", "wb") as f:
    pickle.dump(count_vec, f)

In [ ]:
print("Shape of bags_of_words : %s" % (bags_of_words.shape,))

vocab  = count_vec.get_feature_names()
print("Num of vocab : %s" % (len(vocab)))
print("Sample of vocab : %s" % (vocab[:3]))

In [ ]:
path = './review_vocab.dat'

with open(path, mode='w') as f:
    f.write('\n'.join(vocab))

In [ ]:
vocab

## Upload data to Amazon S3 bucket

データを学習用、テスト用（推論用）に分割して、s3へアップロードします。

In [ ]:
nbags_of_words = min(bags_of_words.shape[0], 10_000) # speed up testing with fewer documents

nbags_of_words_training = int(0.95*nbags_of_words)
nbags_of_words_test = nbags_of_words - nbags_of_words_training

bags_of_words_training = bags_of_words[:nbags_of_words_training]
bags_of_words_test = bags_of_words[nbags_of_words_training:nbags_of_words]

print('training set dimensions = {}'.format(bags_of_words_training.shape))
print('test set dimensions = {}'.format(bags_of_words_test.shape))

ここでは、データをMXNet RecordIO Protobuf形式に変換します。

In [ ]:
%%time

# convert documents_training to Protobuf RecordIO format
recordio_protobuf_serializer = RecordSerializer()
fbuffer = recordio_protobuf_serializer.serialize(bags_of_words_training)

データをアップロードします。

In [ ]:
session = sagemaker.Session()
role = get_execution_role()

bucket = session.default_bucket()
prefix = "sagemaker/DEMO-lda-introduction"

In [ ]:
# upload to S3 in bucket/prefix/train
fname = 'lda.data'
s3_object = os.path.join(prefix, 'train', fname)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_object).upload_fileobj(fbuffer)

s3_train_data = 's3://{}/{}'.format(bucket, s3_object)
print('Uploaded data to S3: {}'.format(s3_train_data))

# Training the LDA Model

Amazon SageMaker LDAは、観測値の集合を異なるトピックの混合物として記述しようとする教師なし学習アルゴリズムです。    

![](https://upload.wikimedia.org/wikipedia/commons/4/4d/Smoothed_LDA.png)

- $M$: 文書数
- $N$: 単語数
- $\alpha$: 文書ごとのトピック分布に対するディリクレ分布のパラメータ
- $\beta$: トピックごとの単語分布に対するディリクレ分布のパラメータ
- $\theta_m$: 文書mのトピック分布
- $\varphi_k$: トピックkの単語分布
- $z_{mn}$: 文書mのn番目の単語の潜在トピック
- $w_{mn}$: 文書mのn番目の単語(観測データ)

パラメータ推定にはtensor spectral decompositionを使用しています。

## Create LDA Container

In [ ]:
# SageMaker LDA Docker container
region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("lda", region_name)

print('Using SageMaker LDA container: {} ({})'.format(container, region_name))

## Set Hyperparameters

SageMaker LDAには以下のハイパーパラメータがあります。

* **`num_topics`** - LDAモデル内のトピックまたはカテゴリの数
    * 通常、これは事前にはわかりません

* **`feature_dim`** - vocabularyのサイズ

* **`mini_batch_size`** - 入力される文書の数

* **`alpha0`** - *(optional)* トピック混合物の「混合度」
  * `alpha0` が小さい場合、文書は1つまたは少数のトピックで表される傾向があります
  * `alpha0` が大きい場合(１より大きい)、文書は複数または多数のトピックの均等な組み合わせになる傾向があります。
  * デフォルト: `alpha0 = 1.0`.
  
 
SageMaker LDAは現在、シングルインスタンスのCPUトレーニングのみをサポートしています。

In [ ]:
ntopics = 20
vocabulary_size = len(vocab)

# specify general training job information
lda = sagemaker.estimator.Estimator(
    container,
    role,
    output_path = 's3://{}/{}/output'.format(bucket, prefix),
    instance_count = 1,
    instance_type = 'ml.c5.2xlarge',
    sagemaker_session = session,
)

# set algorithm-specific hyperparameters
lda.set_hyperparameters(
    num_topics=ntopics,
    feature_dim=vocabulary_size,
    mini_batch_size=nbags_of_words_training,
    alpha0=1.0,
    max_restarts=10,
    max_iterations=1000,
    tol=1e-8
)

## Training

上記の設定でアルゴリズムの実行時間は4-5分です

In [ ]:
# run the training job on input data stored in S3
start = time.time()
try:
    lda.fit({'train': s3_train_data})
except RuntimeError as e:
    print(e)  

end = time.time()
print("Training took", end - start, "seconds")

In [ ]:
print('Training job name: {}'.format(lda.latest_training_job.job_name))

# Evaluating the Model Output

S3からモデルファイルをダウンロードし、検証します。    
モデルは学習時に推定された$\alpha$と$\beta$のパラメータを含む2つの配列で構成されています。

In [ ]:
# download and extract the model file from S3
job_name = lda.latest_training_job.job_name
model_fname = 'model.tar.gz'
model_object = os.path.join(prefix, 'output', job_name, 'output', model_fname)
boto3.Session().resource('s3').Bucket(bucket).Object(model_object).download_file(fname)

with tarfile.open(fname) as tar:
    tar.extractall()
print('Downloaded and extracted model tarball: {}'.format(model_object))

In [ ]:
# obtain the model file
model_list = [fname for fname in os.listdir('.') if fname.startswith('model_')]
model_fname = model_list[0]
print('Found model file: {}'.format(model_fname))

In [ ]:
# get the model from the model file and store in Numpy arrays
alpha, beta = mx.ndarray.load(model_fname)
learned_alpha = alpha.asnumpy()
learned_beta = beta.asnumpy()

print('\nLearned alpha.shape = {}'.format(learned_alpha.shape))
print('Learned beta.shape = {}'.format(learned_beta.shape))

In [ ]:
# visualize alpha
sns.lineplot(x=range(len(learned_alpha)), y=learned_alpha);

In [ ]:
# visualize beta
sns.heatmap(learned_beta, vmax=0.01); # (topics, words)

In [ ]:
for topic_nr in range(ntopics):
    # print most important words for a given topic

    beta = learned_beta[topic_nr]
    idx = np.argsort(beta)

    print("")
    print("Topic", topic_nr)
    print("=====================")
    for i in idx[:-16:-1]:
        print("{:12} {:f}".format(vocab[i], beta[i]))

## Interpretation

トピックの数を20にして各トピックの重要単語を15個出力すると以下のような結果になりました。

```
Topic 0
=====================
チョコレート       0.014655
人気           0.012782
女性           0.012509
クリスマス        0.012506
限定           0.011682
ブランド         0.010410
サイト          0.010372
多い           0.009846
女子           0.009572
アイテム         0.009305
話題           0.009260
cm           0.008218
自分           0.008210
商品           0.008137
関連           0.007626

Topic 1
=====================
映画           0.076599
作品           0.027791
公開           0.024247
監督           0.020890
本作           0.019164
世界           0.016903
映像           0.015342
今回           0.010171
全国           0.009590
サイト          0.009230
movie        0.008435
記事           0.008399
主演           0.008226
公式           0.008088
特集           0.007954

Topic 2
=====================
....
```

Topic0の解釈はいろいろと考えられますが、Topic1は直感的には映画を指しているように思われます。    
num_topicsを変更していろいろ試してみましょう

## Deploy and Inference

`deploy()`関数を使用して推論エンドポイントを作成します。推論を行うインスタンスタイプとインスタンスの初期数を指定します。    

_**NOTE: 実際にサービス上でリアルタイムに使用するためには、文書に対して前処理（トークナイズ、単語の頻度表現）を行った上で推論エンドポイントへリクエストする必要があります。AWS LambdaやSageMakerの推論パイプラインなどを使用することができます**_  

In [ ]:
lda_inference = lda.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m5.xlarge',
)

In [ ]:
print('Endpoint name: {}'.format(lda_inference.endpoint_name))

In [ ]:
# configure data format (CSV, JSON, RECORDIO Protobu)
lda_inference.serializer = CSVSerializer()
lda_inference.deserializer = JSONDeserializer()

In [ ]:
# query endpoint
results = lda_inference.predict(bags_of_words_test[:1])
print(json.dumps(results, sort_keys=True, indent=2))

In [ ]:
# let's predict on the whole test set
results = lda_inference.predict(bags_of_words_test)

In [ ]:
len(results['predictions'])

In [ ]:
# delete endpoint
sagemaker.Session().delete_endpoint(lda_inference.endpoint_name)